### *IPCC SR15 scenario assessment*
<img style="float: right; height: 80px; padding-left: 20px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Final energy demand and carbon intensity analysis

This notebook contains the scripts to generate **Figure 2.14** showing final energy demand development
and the carbon intensity across scenarios
for the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file with scenario categorisation and quantitative indicators can be downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).  
Alternatively, it can be re-created using the notebook `sr15_2.0_categories_indicators`.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r2.0.xlsx')

In [ ]:
sr1p5.load_meta('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream, Loader=yaml.FullLoader)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
years = [2020, 2030, 2050, 2070, 2100]

In [ ]:
df = sr1p5.filter(category=cats, year=years)

## Set specifications for filter and plotting and initialize a data list

In [ ]:
save_name = 'output/fig2.14{}.{}'

In [ ]:
categories = cats.copy()
categories.remove('Above 2C')
column = 'category'

In [ ]:
filter_args = dict(df=sr1p5, category=categories, marker=None, join_meta=True)

In [ ]:
def plotting_args(name, filetype='png', hlines=[]):
    return {'categories': categories, 'column': 'category', 'years': years, 'add_marker': marker,
            'hlines': hlines, 'save': save_name.format(name, filetype)}

In [ ]:
data = []

## Development of final energy demand by category

In [ ]:
fe = df.filter(variable='Final Energy').timeseries()
fe.index = fe.index.droplevel([2, 3, 4])

In [ ]:
name = 'final_energy'
_fe = pyam.filter_by_meta(df.filter(variable='Final Energy').timeseries(), **filter_args)
boxplot_by_cat(_fe, **plotting_args('a_{}'.format(name)),
               ylabel='Final Energy (EJ)')

In [ ]:
data.append(('Final energy', _fe))

## Share of electricity in final demand

In [ ]:
fe_ele = df.filter(variable='Final Energy|Electricity').timeseries()
fe_ele.index = fe_ele.index.droplevel([2, 3, 4])

In [ ]:
fe_ele_share = pyam.filter_by_meta(fe_ele / fe * 100, **filter_args)

In [ ]:
name = 'ele_share'
boxplot_by_cat(fe_ele_share, **plotting_args('c_{}'.format(name)), legend=False,
               ylabel='Electricity share in Final Energy (%)')

In [ ]:
data.append(('Electricity share', fe_ele_share))

## Plot the development of the carbon intensity of electricity vs. the residual energy demand

In [ ]:
co2 = df.filter(variable='Emissions|CO2').timeseries()
co2.index = co2.index.droplevel([2, 3, 4])

In [ ]:
co2_ele = df.filter(variable='Emissions|CO2|Energy|Supply|Electricity').timeseries()
co2_ele.index = co2_ele.index.droplevel([2, 3, 4])

In [ ]:
carbon_intensity_ele = pyam.filter_by_meta(co2_ele / fe_ele, **filter_args)

In [ ]:
name = 'carbon_intensity_electricity'
boxplot_by_cat(carbon_intensity_ele, **plotting_args('b_{}'.format(name), hlines=[0]), legend=False,
               ylabel='Carbon intensity of electricity (gCO2/MJ)')

In [ ]:
data.append(('Carbon intensity of electricity', carbon_intensity_ele))

In [ ]:
carbon_intensity_residual = pyam.filter_by_meta((co2 - co2_ele) / (fe - fe_ele), **filter_args)

In [ ]:
name = 'carbon_intensity_residual'
boxplot_by_cat(carbon_intensity_residual, **plotting_args('d_{}'.format(name), hlines=[0]), legend=False,
               ylabel='Carbon intensity of residual fuel mix (gCO2/MJ)')

In [ ]:
data.append(('Carbon intensity of residual', carbon_intensity_residual))

## Export timeseries data to `xlsx`

In [ ]:
writer = pd.ExcelWriter('output/fig2.14_data_table.xlsx')
for (name, _df) in data:
    pyam.utils.write_sheet(writer, name, _df, index=True)
writer.save()